### 1️⃣ Importaciones y configuración

In [1]:
import sys
import os
from sklearn.preprocessing import StandardScaler

BASE_DIR = os.path.abspath(os.path.join(os.path.dirname("__file__"), "..", ".."))
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

TRAIN_CSV = os.path.join(OUTPUT_DIR, "train_processed.csv")
VAL_CSV   = os.path.join(OUTPUT_DIR, "val_processed.csv")
TEST_CSV  = os.path.join(OUTPUT_DIR, "test_processed.csv")

sys.path.insert(0, BASE_DIR)

from src.part_2.main import run_training
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.neural_network import MLPRegressor
import pandas as pd
from sklearn.linear_model import LinearRegression


### 2️⃣ Funciones de evaluación de métricas

In [2]:
def mape(y_true, y_pred):
    """Mean Absolute Percentage Error"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape_val = mape(y_true, y_pred)
    return {
        "MSE": float(mse),
        "MAE": float(mae),
        "MAPE": float(mape_val)
    }


### 3️⃣ Funciones de entrenamiento de los modelos BP-F y MLR-F

In [7]:
def train_bp_f(X_train, y_train, X_test, activation="identity"):
    """
    BP-F model using scikit-learn's MLPRegressor
    """
    mlp = MLPRegressor(
        hidden_layer_sizes=(20, 10),
        activation=activation,
        solver="adam",
        max_iter=200,
        tol=1e-4,
        random_state=42
    )
    mlp.fit(X_train, y_train)
    preds = mlp.predict(X_test)
    return preds, mlp

def train_mlr_f(X_train, y_train, X_test):
    """
    MLR-F model using scikit-learn's LinearRegression
    """
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    preds = lr.predict(X_test)
    return preds, lr


### 4️⃣ Cargar datos usando las variables del notebook


In [8]:
df_train = pd.read_csv(TRAIN_CSV)
df_test  = pd.read_csv(TEST_CSV)

# Drop PID and separate features and target
X_train = df_train.drop(columns=["SalePrice", "PID"])
y_train = df_train["SalePrice"]

X_test  = df_test.drop(columns=["SalePrice", "PID"])
y_test  = df_test["SalePrice"]



### 5️⃣ Feature Scaling


In [19]:
activation_function = "linear"  # Example activation function ["linear", "sigmoid", "tanh", "relu"]

# Scale features
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled  = scaler_X.transform(X_test)

if activation_function == "tanh" or activation_function == "sigmoid":
    y_train_transformed = np.log1p(y_train)
else:
    y_train_transformed = y_train


### 5️⃣ Entrenamiento y predicción de los modelos

In [27]:
# BP Custom Neural Network

nn, pid_test, y_test_values, preds_bp, trainer = run_training(activation_function, verbose=True, CSV_TRAIN=TRAIN_CSV, CSV_VAL=VAL_CSV, CSV_TEST=TEST_CSV)

print("BP Custom:")
bp_custom_evaluation = evaluate_model(y_test_values, preds_bp)


Training Neural Network...
     Epoch  Train Error  Val Error
0        1     0.501342   0.418569
1        2     0.477339   0.398607
2        3     0.454774   0.379885
3        4     0.433558   0.362325
4        5     0.413608   0.345854
5        6     0.394846   0.330404
6        7     0.377198   0.315909
7        8     0.360596   0.302310
8        9     0.344976   0.289549
9       10     0.330276   0.277574
10      11     0.316441   0.266335
11      12     0.303417   0.255786
12      13     0.291154   0.245882
13      14     0.279606   0.236584
14      15     0.268728   0.227853
15      16     0.258481   0.219653
16      17     0.248824   0.211951
17      18     0.239723   0.204714
18      19     0.231142   0.197915
19      20     0.223051   0.191524
20      21     0.215420   0.185517
21      22     0.208220   0.179869
22      23     0.201425   0.174557
23      24     0.195011   0.169561
24      25     0.188955   0.164860
25      26     0.183235   0.160435
26      27     0.177830   0.

In [24]:
# BP-F Model
activation_map_bp_f = {
    "linear": "identity",
    "sigmoid": "logistic",
    "tanh": "tanh",
    "relu": "relu"
}

activation_bp_f = activation_map_bp_f[activation_function]

preds_bp_f, bp_f_model = train_bp_f(X_train_scaled, y_train_transformed, X_test_scaled, activation_bp_f)

print("BP-F:")
bpf_evaluation = evaluate_model(y_test, preds_bp_f)


/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


BP-F:


/opt/homebrew/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [25]:
# MLR-F Model
preds_mlr_f, mlr_model = train_mlr_f(X_train, y_train, X_test)

print("MLR-F:")
mlfr_evaluation = evaluate_model(y_test, preds_mlr_f)

MLR-F:


/opt/homebrew/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


In [28]:
# Comparative Metrics
metrics = pd.DataFrame({
    "BP Custom": bp_custom_evaluation,
    "BP-F": bpf_evaluation,
    "MLR-F": mlfr_evaluation
}).T

display(metrics)


,MSE,MAE,MAPE
BP Custom,5.206197e+08,16585.978515,9.674769
BP-F,3.084110e+10,173407.835776,110.035904
MLR-F,4.418003e+08,13784.441236,8.424560
